<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A7%88%EB%8B%98_%EC%A0%9C3%EC%9C%A0%ED%98%95_AI%EB%AC%B8%EC%A0%9C%ED%92%80%EC%9D%B4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# 랜덤 시드 고정
np.random.seed(42)

# 데이터셋 1: 정규성 및 등분산 검정용
data1 = {
    'group_A': np.random.normal(100, 10, 50),
    'group_B': np.random.normal(100, 20, 50),
    'group_C': np.random.exponential(scale=10, size=50) # 비정규분포
}
df_1 = pd.DataFrame(data1)

# 데이터셋 2: 범주형 데이터 (카이제곱, 피셔)
data2 = {
    'Gender': ['Male']*30 + ['Female']*20 + ['Male']*10 + ['Female']*40,
    'Preference': ['Like']*25 + ['Dislike']*25 + ['Like']*5 + ['Dislike']*45
}
df_2 = pd.DataFrame(data2)

# 데이터셋 3: 소규모 범주형 데이터 (피셔 정확검정용)
data3 = {
    'Treatment': ['Drug']*10 + ['Placebo']*10,
    'Outcome': ['Cured']*8 + ['Not']*2 + ['Cured']*2 + ['Not']*8
}
df_3 = pd.DataFrame(data3)

# 데이터셋 4: 다집단 분석 (ANOVA, 사후검정, 비모수)
g1 = np.random.normal(70, 5, 30)
g2 = np.random.normal(75, 5, 30)
g3 = np.random.normal(80, 5, 30)
g4 = np.random.exponential(10, 30) # 비정규
g5 = np.random.normal(70, 15, 30) # 등분산 깨짐

df_4 = pd.DataFrame({
    'score': np.concatenate([g1, g2, g3]),
    'group': ['A']*30 + ['B']*30 + ['C']*30
})

df_5 = pd.DataFrame({
    'score': np.concatenate([g1, g4, g5]),
    'group': ['A']*30 + ['B']*30 + ['C']*30
})

df_6 = pd.DataFrame({
    'score': np.concatenate([g1, g2, g5]), # 정규성 만족 가정, 등분산 불만족
    'group': ['A']*30 + ['B']*30 + ['C']*30
})

In [13]:
#1
from scipy import stats
res = stats.anderson(df_1['group_A'])
print('검정 통계량: ', res[0])
print('임계값: ', res[1][2])

검정 통계량:  0.19069369484738274
임계값:  0.736


In [19]:
#2
from scipy import stats
s,p = stats.bartlett(df_1['group_A'], df_1['group_B'])
print(round(s,2), format(p,'.10f')) #등분산 하지 않다.

17.96 0.0000225335


In [17]:
#3
from scipy import stats
s,p = stats.fligner(df_1['group_A'], df_1['group_C'])
print(round(p,4)) #등분산 하다.

0.1711


In [25]:
#4
from scipy import stats

df_c = pd.crosstab(df_2['Gender'], df_2['Preference'])

s,p,d,exp = stats.chi2_contingency(df_c)
print(round(s,2),format(p,'.10f'),d)
print(exp)

60.76 0.0000000000 1
[[42. 18.]
 [28. 12.]]


In [38]:
#5
from scipy import stats
#교차표 생성
df_m = pd.crosstab(df_3['Treatment'], df_3['Outcome'])

#카이제곱 검정
#s,p,d,exp = stats.chi2_contingency(df_m)
#print(exp) #기대 빈도 5이하인 셀 100%

#fisher_exact()
s,p = stats.fisher_exact(df_m)
print(s,round(p,4))

16.0 0.023


In [56]:
#6
from statsmodels.formula.api import ols
import statsmodels.api as sm
model = ols('score~C(group)',data = df_4).fit()
res = sm.stats.anova_lm(model)
print(res)
print() #p-value < 0.05 -> 적어도 하나의 평균에 차이가 있음

from statsmodels.stats.multicomp import pairwise_tukeyhsd
res = pairwise_tukeyhsd(df_4['score'], df_4['group'], alpha=0.05)
print(res)
print(round(res.meandiffs[1], 2))

            df       sum_sq     mean_sq          F        PR(>F)
C(group)   2.0  1807.261909  903.630954  35.653702  4.906640e-12
Residual  87.0  2204.985432   25.344660        NaN           NaN

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
     A      B   5.3332 0.0003 2.2337  8.4327   True
     A      C  10.9751    0.0 7.8756 14.0746   True
     B      C   5.6419 0.0001 2.5424  8.7414   True
---------------------------------------------------
10.98


In [80]:
#7
from statsmodels.sandbox.stats.multicomp import MultiComparison
comp = MultiComparison(df_4['score'], df_4['group'])
res = comp.allpairtest(scipy.stats.ttest_ind, method='bonf')
# print(res[0].data)
res = pd.DataFrame(res[0].data)
print(res.iloc[3, 4])

0.0004


In [85]:
#8
from scipy import stats
s,p = stats.mannwhitneyu(df_1['group_A'], df_1['group_C'])
print(s,format(p,'.10f')) #0.0000 : 둘의 평균엔 차이가 있다.

2500.0 0.0000000000


In [89]:
pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 9.6 MB/s eta 0:00:00


In [96]:
#9
import pingouin as pg
res = pg.welch_anova(data = df_6, dv = 'score', between='group')
print(round(res.iloc[0, -2], 4))

0.0001


In [101]:
#10
from scipy import stats
df_A = df_5[df_5['group'] == 'A']['score']
df_B = df_5[df_5['group'] == 'B']['score']
df_C = df_5[df_5['group'] == 'C']['score']

s,p = stats.kruskal(df_A, df_B, df_C)
print(round(s,2), format(p,'.10f')) #평균이 다른게 적어도 하나 있다.


59.27 0.0000000000


In [103]:
import pandas as pd
import numpy as np

data = {
    'Store': ['Gangnam', 'Hongdae', 'Itaewon', 'Gangnam', 'Hongdae'],
    'Manager': ['Kim', 'Lee', 'Park', 'Choi', 'Jung'],
    'Product': ['Americano', 'Latte', 'Tea', 'Latte', 'Americano'],
    'Sales_01': [120, 150, 80, 200, 90],
    'Sales_02': [110, 160, 90, 190, 95],
    'Sales_03': [130, 155, 85, 210, 100],
    'Sales_04': [140, 165, 95, 220, 105]
}
df = pd.DataFrame(data)

In [104]:
print(df)

     Store Manager    Product  Sales_01  Sales_02  Sales_03  Sales_04
0  Gangnam     Kim  Americano       120       110       130       140
1  Hongdae     Lee      Latte       150       160       155       165
2  Itaewon    Park        Tea        80        90        85        95
3  Gangnam    Choi      Latte       200       190       210       220
4  Hongdae    Jung  Americano        90        95       100       105


In [110]:
#1
res1 = pd.melt(df, id_vars=['Store','Manager','Product'])
#2
res2 = pd.melt(df, id_vars = ['Store','Manager','Product'], var_name = 'Month', value_name='Amount')
#3
res3 = pd.melt(df, id_vars = ['Store','Manager','Product'],value_vars=['Sales_01','Sales_02'],
        var_name='Month', value_name='Amount')
#4
res2 = pd.melt(df, id_vars = ['Store','Manager','Product'], var_name = 'Month', value_name='Amount')
res2['Month'] = res2['Month'].str[-2:]
print(res2[res2['Month'] == '03']['Amount'].mean())


문제 5. 필터링 후 집

 모든 월별 매출을 녹여낸 후(Long Format), 'Product'가 'Americano'인 데이터만 필터링하시오. 필터링된 데이터의 매출(value) 총합을 구하시오.

     Store Manager    Product  Sales_01  Sales_02  Sales_03  Sales_04
0  Gangnam     Kim  Americano       120       110       130       140
1  Hongdae     Lee      Latte       150       160       155       165
2  Itaewon    Park        Tea        80        90        85        95
3  Gangnam    Choi      Latte       200       190       210       220
4  Hongdae    Jung  Americano        90        95       100       105
890


In [161]:
#5
res5 = pd.melt(df, id_vars=['Store','Manager','Product'])
res5 = res5[res5['Product'] == 'Americano']['value'].sum()
print(res5)

#6
res6 = pd.melt(df, id_vars = ['Store','Manager','Product'])
print(res6.groupby('Store')['value'].sum().idxmax())

#7
#결측치 있다고 가정하기
import numpy as np
df.iloc[2, -1] = np.nan

print(pd.melt(df, id_vars=['Store','Manager','Product'])) #결측치 옵션이 어딨는데,,

Gangnam
      Store Manager    Product  variable  value
0   Gangnam     Kim  Americano  Sales_01  120.0
1   Hongdae     Lee      Latte  Sales_01  150.0
2   Itaewon    Park        Tea  Sales_01   80.0
3   Gangnam    Choi      Latte  Sales_01  200.0
4   Hongdae    Jung  Americano  Sales_01   90.0
5   Gangnam     Kim  Americano  Sales_02  110.0
6   Hongdae     Lee      Latte  Sales_02  160.0
7   Itaewon    Park        Tea  Sales_02   90.0
8   Gangnam    Choi      Latte  Sales_02  190.0
9   Hongdae    Jung  Americano  Sales_02   95.0
10  Gangnam     Kim  Americano  Sales_03  130.0
11  Hongdae     Lee      Latte  Sales_03  155.0
12  Itaewon    Park        Tea  Sales_03   85.0
13  Gangnam    Choi      Latte  Sales_03  210.0
14  Hongdae    Jung  Americano  Sales_03  100.0
15  Gangnam     Kim  Americano  Sales_04  140.0
16  Hongdae     Lee      Latte  Sales_04  165.0
17  Itaewon    Park        Tea  Sales_04    NaN
18  Gangnam    Choi      Latte  Sales_04  220.0
19  Hongdae    Jung  Americano  

In [173]:
#8
#pd.melt(df, id_vars=['Store','Product']) #variable에 Manager가 들어감.
res8 = pd.melt(df, id_vars=['Store','Product'], value_vars = ['Sales_01','Sales_02','Sales_03','Sales_04']) #이렇게 해야함

#9
res2 = pd.melt(df, id_vars = ['Store','Manager','Product'], var_name = 'Month', value_name='Amount')
res2_p = res2.pivot_table(index = ['Store','Manager','Product'], columns=['Month'], values=['Amount'])
print(res2_p.isnull().sum().sum())

#10
res10 = pd.melt(df, id_vars = ['Store','Manager','Product'], var_name='Month', value_name='Amount')
res10 = res10.sort_values(by=['Month','Amount'], ascending=[False, True])
print(res10.iloc[0,-1])

1
105.0
